In [2]:
#Ensure we are running Python 3
import sys
pyinfo = sys.version_info
assert (pyinfo.major == 3), "Python 3 required. The version you are using is: {}".format(pyinfo) 

In [3]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

In [429]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def colorspace(img, c_space):
    return cv2.cvtColor(img, c_space)

def get_channel(img, c):
    return img[:, :, c]

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def drawROI(img, x1, y1, x2, y2):
    out = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0))
    return out

def drawPolygon(img):
    pts = np.array([[540,450],[740,450],[1180,720],[100,720]], np.int32)
    pts = pts.reshape((-1,1,2))
    out = cv2.polylines(img,[pts],True,(0,255,255))
    return out

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

run your solution on all test_images and make copies into the test_images directory).

In [430]:
#Adaptive Binary Threshold.
#We use R and G channels from RGB, V from HSV and S from HLS.
#We iteratively find threshold parameters that give enough pixels but not too many

def get_ROI(img, x1, y1, x2, y2):
    return img[y1:y2, x1:x2, :]

#Binary Thresholding on on ROI image. 
# input ROI image is 3 channel
# returns a 3channel Binary image (0 and 255)
def binary_threshold(roi, init_thresh = 200, minRG = 85, thresh_dt = 5, bailout = 20):
    
    thresh_img = np.zeros_like(roi[:, :, 0])
    total_pixels = thresh_img.shape[0]*thresh_img.shape[1]
    thresh = init_thresh
    
    #Using R and G channels to mask out dark or grey road/ shadow values
    (B,G,R) = cv2.split(roi)
    
    R_dx = np.absolute(cv2.Sobel(R, cv2.CV_64F, 1, 0))
    R_dx *= 255/np.max(R_dx)
    
    hls = colorspace(roi, cv2.COLOR_BGR2HLS)
    S = get_channel(hls, 2)
    
    hsv = colorspace(roi, cv2.COLOR_BGR2HSV)
    V = get_channel(hsv, 2)
    
    #Create an initial binary image. We will refine this if we don't have enough pixels or too many pixels
    thresh_img[(V > thresh) | ((R > thresh) & (G > thresh)) | (S > thresh) ] = 255
    thresh_img[(R < minRG) & (G < minRG)] = 0
    
    #Count of Non-Zero mask pixels
    nzcount = np.count_nonzero(thresh_img)
    
    #Bailout Counter. If we can not reach a good value in n steps, stop wasting time and move on.
    counter = 0
    #We want the number of lane pixels to be within 1% to 3% of the total pixels in image
    while (nzcount < 0.01*total_pixels) | (nzcount > 0.03*total_pixels) & (thresh > 100) & (thresh < 231):
        counter += 1
        if (counter == bailout):
            print("Unable to find a good value in 10 steps. Bailing out!")
            thresh_img = np.zeros_like(thresh_img)
            break
        if (nzcount < 0.02*total_pixels):
            thresh -= thresh_dt
        else:
            thresh += thresh_dt
        thresh_img = np.zeros_like(thresh_img)
        thresh_img[(V > thresh) | ((R > thresh) & (G > thresh)) | (S > thresh) ] = 255
        thresh_img[(R < minRG) & (G < minRG)] = 0
        nzcount = np.count_nonzero(thresh_img)
        
    bin_img = np.dstack((thresh_img, thresh_img, thresh_img))
    return bin_img

#read saved calibration and distortion



In [461]:
def warp(img, src_pts, dst_pts):
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    out = cv2.warpPerspective(img, M, (640, 480))
    return out

In [462]:
"""
Run the processing pipeline on image
if return_at == -1, all steps are perfomed, else, steps are performed till the required number
"""
def processImage(image, return_at = -1):
    
    Y,X,C = image.shape
    lines = None
    
    #undistort image
    #dst_img = cv2.undistort(image, mtx, dist, None, mtx)
    
    #extract image ROI
    #Most parts of the image do not provide useful info
    #roi = drawROI(image, *(200, 400), *(1080, 720)) 
    ROI_x1, ROI_y1, ROI_x2, ROI_y2 = 200, 450, 1080, 720 
    roi = get_ROI(image, ROI_x1, ROI_y1, ROI_x2, ROI_y2)
    
    out_image = image
    bin_img = binary_threshold(roi)
    
    #weighted_image = cv2.addWeighted(bin_img, 0.8, roi, 0.2, 0)
    
    out_image = np.zeros_like(image)
    out_image[ROI_y1:ROI_y2, ROI_x1:ROI_x2, :] = bin_img
   
    #out_image = drawPolygon(out_image)
    
    src =  np.array([[585,450],[695,450],[1180,720],[100,720]], dtype=np.float32)
    offset = 20
    dst = np.array([[offset, offset], [640-offset, offset], [640-offset,480-offset], [offset, 480-offset]], dtype=np.float32)
    out_image = warp(out_image, src, dst)
    return lines, out_image

In [463]:
img_dir = "test_images"
for img_name in os.listdir(img_dir):
    #ONly look at image files:
    if os.path.isfile(os.path.join("test_images", img_name)):
        print(img_name)
        img_in_path = os.path.join("test_images", img_name);
        img_out_path = os.path.join("test_images", "output", img_name);
        image = mpimg.imread(img_in_path)
        lines, out_image = processImage(image, -1)
        mpimg.imsave(img_out_path, out_image)
        
print("Done")

straight_lines1.jpg
challenge_4.jpg
test6.jpg
test2.jpg
test4.jpg
straight_lines2.jpg
test3.jpg
challenge_2.jpg
test5.jpg
challenge_1.jpg
test1.jpg
challenge_5.jpg
challenge_3.jpg
Done


In [464]:
# Import everything needed to edit/save/watch video clips
import imageio.plugins
imageio.plugins.ffmpeg.download()
import moviepy.editor as mp
from moviepy.editor import VideoFileClip
from IPython.display import HTML



In [465]:
def process_image(image, return_at=-1):#(return_at, image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    _, result = processImage(image, return_at)
    return result

In [466]:
import tqdm
white_output = 'project_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_out.mp4
[MoviePy] Writing video project_out.mp4


100%|█████████▉| 1260/1261 [00:39<00:00, 31.85it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_out.mp4 

CPU times: user 1min 48s, sys: 3.26 s, total: 1min 51s
Wall time: 39.8 s


In [467]:
yellow_output = 'challenge_out.mp4'
clip2 = VideoFileClip('challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video challenge_out.mp4
[MoviePy] Writing video challenge_out.mp4


100%|██████████| 485/485 [00:20<00:00, 23.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_out.mp4 

CPU times: user 1min 3s, sys: 1.65 s, total: 1min 5s
Wall time: 20.9 s


In [468]:
yellow_output = 'harder_out.mp4'
clip2 = VideoFileClip('harder_challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video harder_out.mp4
[MoviePy] Writing video harder_out.mp4


100%|█████████▉| 1199/1200 [01:05<00:00, 16.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_out.mp4 

CPU times: user 3min 18s, sys: 5.16 s, total: 3min 24s
Wall time: 1min 5s
